<a href="https://colab.research.google.com/github/juanpblgomez/mentoria-diplodatos-2024-analisis-cursos-previsionales/blob/main/ayvd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctico 1 - Análisis y Visualización de Datos

Trabajaremos sobre los datos anonimizados de compras de una plataforma de cursos onlines. Exploraremos las distintas fuentes de datos y haremos un análisis descriptivo y visual de los datos para entender cómo están distribuidas las ventas.

El dataset está constituido por tres bases: clientes, ventas y contactos. Hay diversos análisis que se pueden hacer a partir de las mismas: segmentación de clientes, análisis de texto de las consultas, análisis de series de tiempo de las ventas, etc.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import seaborn

#### Lectura de los datasets

In [10]:
def read_from_raw(github_url):
  raw_url = github_url.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/")
  return pd.read_csv(raw_url, encoding = "ISO-8859-1")

In [11]:
df_contacts = read_from_raw('https://github.com/juanpblgomez/mentoria-diplodatos-2024-analisis-cursos-previsionales/blob/main/datos/contacts_anon.csv')
df_contacts[:3]

,id,fecha,nombre,email,genero,asunto,mensaje
0,80242452935422199706,16/03/2024 14:55:28,Dani Castellanos Briones,dani.castellanos.briones2358@hotmail.com,M,Consulta general,. quisiera saber si existe algun simulador del...
1,15409661513166318845,13/03/2024 16:13:09,Apolinar Jimenez Teruel,apolinar.jimenez.teruel6338@gmail.com,M,Consulta general,no puedo ver los videos del curso. necesito ayuda
2,67687345747217393942,12/03/2024 18:47:42,Evangelina Clotilde Requena Sans,evangelina.clotilde.requena.sans5853@gmail.com,F,Consulta general,"hola, me surge una duda de la ley . mujer de a..."


In [12]:
df_customers = read_from_raw('https://github.com/juanpblgomez/mentoria-diplodatos-2024-analisis-cursos-previsionales/blob/main/datos/customers_anon.csv')
df_customers[:3]

,id,nombre,email,genero
0,87383708928538182091,Sancho Quesada,sancho.quesada631@gmail.com,M
1,90963905505013869370,Rosario Donoso,rosario.donoso4687@hotmail.com,F
2,36171761627209661626,Dani Castellanos Briones,dani.castellanos.briones2358@hotmail.com,M


In [13]:
df_orders = read_from_raw('https://github.com/juanpblgomez/mentoria-diplodatos-2024-analisis-cursos-previsionales/blob/main/datos/orders_anon.csv')
df_orders[:3]

,número,Fecha,moneda,monto total,cant. de productos,estado del pago,medio de pago,estado de la entrega,forma de entrega,envío: dirección,envío: código postal,producto: código / sku,producto: cantidad,producto: precio,código país,código área,nombre,email,producto,codigo_promocional
0,5285,17/03/2024 03:29:27,ARS,20280.0,1,Pagado,Mercadopago,NaN,NaN,NaN,NaN,NaN,1,20280.0,54.0,NaN,Sancho Quesada,sancho.quesada631@gmail.com,Curso 2,NaN
1,5284,16/03/2024 14:33:27,ARS,20280.0,1,Pagado,Mercadopago,NaN,NaN,NaN,NaN,NaN,1,20280.0,54.0,297.0,Dani Castellanos Briones,dani.castellanos.briones2358@hotmail.com,Curso 2,NaN
2,5283,15/03/2024 19:21:18,ARS,20280.0,1,Pendiente,Mercadopago,NaN,NaN,NaN,NaN,NaN,1,20280.0,54.0,351.0,María Saldaña,maria.saldana6060@gmail.com,Curso 2,NaN
